## Why is text data failiing?

In [96]:
import pandas as pd
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [118]:
steam=pd.read_csv("preprocess_project.csv")
originalSteam=pd.read_csv("steam.csv")
steamDescriptions=pd.read_csv("steam_description_data.csv")
TFIDFDescriptions=pd.read_csv("TFIDFDescriptions.csv")

BOWDescriptions=pd.read_csv("BOWDescriptions.csv")

steam.head()
TFIDFDescriptions.head()

,popularity,action,adventure,available,based,battle,best,build,challenge,challenging,...,try,unique,unlock,use,using,various,want,way,weapons,world
0,Popular,0.225381,0.0,0.0,0.224651,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.163636
1,Popular,0.204416,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.346357,0.0,0.0,0.0,0.0,0.0,0.0,0.210029,0.000000
2,Popular,0.000000,0.0,0.0,0.369677,0.0,0.0,0.0,0.0,0.0,...,0.0,0.314201,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,Popular,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,Popular,0.395474,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [119]:
(df_train, df_test) = train_test_split(TFIDFDescriptions, random_state=0)
X_train = df_train.drop("popularity", axis=1)
X_test = df_test.drop("popularity", axis=1)
y_train = df_train.popularity
y_test = df_test.popularity



In [120]:
(bow_train, bow_test) = train_test_split(BOWDescriptions , random_state=0)
Xb_train = bow_train.drop("popularity", axis=1)
Xb_test = bow_test.drop("popularity", axis=1)
yb_train = bow_train.popularity
yb_test = bow_test.popularity

In [13]:
grid = {'n_estimators':[10,100,200],'max_depth':np.arange(1,30)}
rf = RandomForestClassifier(n_jobs=-1)
rfCV = GridSearchCV(rf,param_grid=grid,n_jobs=-1)
rfCV.fit(X_train,y_train)
print('Random Forests:')
print('optimal parameters:',rfCV.best_params_)
print('optimal validation R2 =',rfCV.best_score_.round(3))

Random Forests:
optimal parameters: {'max_depth': 28, 'n_estimators': 100}
optimal validation R2 = 0.695


In [15]:
bestForest=rfCV.best_estimator_
test_score = bestForest.score(X_test, y_test).round(3)
test_score

0.698

In [20]:
y_test.value_counts()

Low popularity    4658
Average           1166
Popular            945
Name: popularity, dtype: int64

In [17]:
yh = rfCV.predict(X_test)

In [18]:
y = y_test.astype('category')
cols = y.cat.categories
M = confusion_matrix(y, yh)
ConfusionMatrix = pd.DataFrame(M, index=cols, columns=cols)
ConfusionMatrix['total'] = ConfusionMatrix.sum(axis=1)
ConfusionMatrix

,Average,Low popularity,Popular,total
Average,7,1128,31,1166
Low popularity,5,4643,10,4658
Popular,5,865,75,945


In [ ]:
# From the confusion matrix,
#it seems like the mode couldn't really distinguish between what makes a game popular, average and unpopular from text data


In [89]:
# example where this fail 
def predict(appid, model, data, steam):
    data["appid"]=steam["appid"]
    sample=data.loc[data["appid"]==appid]
    sample=sample.drop("popularity",axis=1)
    return model.predict(sample.iloc[:,:-1])

In [92]:
sample=originalSteam.loc[originalSteam["popularity"]=="Popular"].iloc[1,:]
sample

appid                                                                50
name                                          Half-Life: Opposing Force
release_date                                                 1999-11-01
english                                                               1
developer                                              Gearbox Software
publisher                                                         Valve
platforms                                             windows;mac;linux
required_age                                                          0
categories          Single-player;Multi-player;Valve Anti-Cheat enabled
genres                                                           Action
steamspy_tags                                         FPS;Action;Sci-fi
achievements                                                          0
positive_ratings                                                   5250
negative_ratings                                                

In [93]:
appid=sample.appid

In [94]:
pd.set_option('display.max_colwidth', None)
print(str(steamDescriptions.loc[steamDescriptions["steam_appid"]==appid].detailed_description))

4    Return to the Black Mesa Research Facility as one of the military specialists assigned to eliminate Gordon Freeman. Experience an entirely new episode of single player action. Meet fierce alien opponents, and experiment with new weaponry. Named 'Game of the Year' by the Academy of Interactive Arts and Sciences.
Name: detailed_description, dtype: object


In [95]:
predict(appid, bestForest, TFIDFDescriptions, originalSteam)
# but this game should be highly popular.

array(['Low popularity'], dtype=object)

In [121]:
grid = {'learning_rate':[0.01,0.001],'n_estimators':[100,200]}
gbt = GradientBoostingClassifier()
gbtCV = GridSearchCV(gbt,param_grid=grid,n_jobs=-1)
gbtCV.fit(X_train,y_train)
print('Gradient Boosting Trees:')
print('optimal parameters:',gbtCV.best_params_)
print('optimal validation R2 =',gbtCV.best_score_.round(3))

Gradient Boosting Trees:
optimal parameters: {'learning_rate': 0.01, 'n_estimators': 200}
optimal validation R2 = 0.692


In [122]:
bestGbt=gbtCV.best_estimator_
bestGbt.score(X_test, y_test)

0.6941941202540995

In [125]:
yh = bestGbt.predict(X_test)
y = y_test.astype('category')
cols = y.cat.categories
M = confusion_matrix(y, yh)
ConfusionMatrix = pd.DataFrame(M, index=cols, columns=cols)
ConfusionMatrix['total'] = ConfusionMatrix.sum(axis=1)
ConfusionMatrix

,Average,Low popularity,Popular,total
Average,0,1153,13,1166
Low popularity,0,4649,9,4658
Popular,0,895,50,945


In [126]:
predict(appid, bestGbt, TFIDFDescriptions, originalSteam)

array(['Low popularity'], dtype=object)